<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#This-implementation-of-PROSAIL" data-toc-modified-id="This-implementation-of-PROSAIL-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>This implementation of PROSAIL</a></span></li><li><span><a href="#Sensitivity-analysis" data-toc-modified-id="Sensitivity-analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Sensitivity analysis</a></span></li><li><span><a href="#Angular-effects" data-toc-modified-id="Angular-effects-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Angular effects</a></span><ul class="toc-item"><li><span><a href="#Questions:" data-toc-modified-id="Questions:-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Questions:</a></span></li></ul></li><li><span><a href="#A-trip-to-RED/NIR-space" data-toc-modified-id="A-trip-to-RED/NIR-space-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>A trip to RED/NIR space</a></span></li><li><span><a href="#Exploring-the-MTCI-(MERIS-Terrestrial-Chlorophyll-Index)" data-toc-modified-id="Exploring-the-MTCI-(MERIS-Terrestrial-Chlorophyll-Index)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Exploring the MTCI (MERIS Terrestrial Chlorophyll Index)</a></span></li><li><span><a href="#So-what-have-we-learned?" data-toc-modified-id="So-what-have-we-learned?-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>So what have we learned?</a></span></li></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

from prosail_functions import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ipywidgets as widgets



Bad key keymap.all_axes in file matplotlibrc, line 398 ('keymap.all_axes : a                 # enable all axes')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.7.1/matplotlibrc.template
or from the matplotlib source distribution


<div style="float:right">
    <table>
    <tr>
        <td> <img src="figs/nceo_logo.png" alt="Drawing" style="width:200px;height:40px;"/> 
        <td> <img src="figs/multiply_logo.png" alt="Drawing" style="width:40px;height:40px;"/>
    </tr>
    </table>
</div>


# Exploring a canopy RT model: PROSAIL

**J Gómez-Dans (UCL & NCEO)**


## This implementation of PROSAIL

There are quite a few different versions of the SAIL model. This is probably the second simplest, as it assumes a single canopy layer, with a simple hotspot correction. The soil boundary is assumed to be Lambertian, and a combination of two soil spectra (typically one wet and one dry). Leaf optical properties are calculated with **PROSPECT D** (which you should already be familiar with). In total, we have 12 input parameters (as well as the illumination/acquisition geometry, controlled by the solar zenith angle, the view zenith angle and the relative azimuth angle). The input parameters are

* $N$ (leaf layers)
* $Cab$ (chlorophyll ab conc)
* $Car$ (carotenoid conc)
* $Cbrown$ (senescent pigment
* $Cw$ (equivalent leaf water)
* $Cm$ (dry matter conc)
* $C_{ant}$ (anthocyanin concentration)
* LAI (leaf area index)
* LIDF (the mean leaf angle)
* RSOIL (soil brightness term)
* PSOIL (soil moisture factor: 0 wet, 1 dry)
* HOTSPOT (the value of the hotspot parameter, typically the ratio of leave size to canopy height)

The soil is assumed Lambertian, and made up of the linear combination of two spectra, $\rho_{s}^{d}$ and $\rho_{s}^{w}$, dry and wet, respectively. The soil spectrum is calculated as

$$
\rho_{s}=R\cdot\left(P\cdot\rho_{s}^{d} + \left(1-P\right)\cdot \rho_{s}^{w}\right).
$$

This version of PROSAIL uses a Campbell leaf angle distribution function. Rather than choosing discrete distributions, the Campbell function parameterises the LAD function with a single parameter, the mean leaf angle.

## Sensitivity analysis

The aim of this exercise is to look at the sensitivity of PROSAIL to different parametes spectrally, in a way that is similar to what you did before with PROSPECT. Remember that this experiment is still a local sensitivity analysis experiment around $\vec{x}_{0}$, so make sure you consider different locations in parameter space. Also consider the effect of acquisition geometry.

You can use the following GUI to play around with defining $\vec{x}_{0}$. 

* Where do parameters show the most/the least sensitivity spectrally?
* Where do these sensitivities change with acquisition geometry?
* What is the meaning of P=0 and R=0 for the soil parameterisation? Do you see that in the sensitivity plots?
* What happens in the visible part of the spectrum?
* What might cause confusion in interpreting any one portion of the spectrum, or just a few bands?
* What's surprising about the value of the sensitivity of reflectance to soil for high (>5) LAI and `psoil` and `rsoil`.




In [2]:
prosail_sensitvitiy_gui()

Output()

## Angular effects

An often ignored effect is that of acquisition geometry. In this next plot, we'll look at a plot of reflectance of a fairly optically thick canopy (LAI=4) with the sun (`theta`) at some position, and over the principal solar plane. We show the result of simulating both the NIR and RED bands (865 and 650 nm, respectively) using SAIL. You can also tweak the value of the `h` (hotspot) parameter.

### Questions:

* What shape do you see in the angular plot?
* Can you relate that shape or position to the two parameters?
* What would the effect be of not taking into account these angular information?
* In practice, what might make the acquisition geometry change for, e.g.
    * Sentinel 2/Landsat
    * Sentinel 3/OLCI or MODIS
 

In [3]:
widgets.interact(hspot, 
                h=widgets.FloatLogSlider(min=-3, max=0, value=0.1),
                theta=widgets.FloatSlider(min=0, max=70, value=30.))


interactive(children=(FloatLogSlider(value=0.1, description='h', max=0.0, min=-3.0), FloatSlider(value=30.0, d…

<function prosail_functions.hspot(h, theta)>

## A trip to RED/NIR space

Perhaps the first thing you've heard about optical remote sensing is about the use of $NDVI$, the normalised difference vegetation index. This index has a long history (it's 40 years old in 2019!) in remote sensing, and one could say that it is broadly related to the amount of green vegetation. $NDVI$ is calculated using the red and NIR bands (typically, wavelenghts around 680 and 865 nm):

$$
NDVI=\frac{NIR-RED}{NIR+RED}.
$$
As it is a normalised quantity, NDVI goes between -1 and 1, but for vegetation, we usually find that it goes from $\sim$ 0.2 to 0.9.

* $NDVI$ has been correlated to most things. Based on the sensitivity analysis you performed above, can you suggest some of these correlates?

* $NDVI$ is often used as a proxy for LAI. You can use the provided function to explore the relationship beween the VI and LAI. What are your observations?
* You can set a number of extra *nuisance* parameters to vary randomly. Discuss how the value of these different parameters affects the value of the calculated NDVI, and how it affects your retrieval of e.g. LAI. Relate this to your sensitivity analysis.
* You can also change the soil brightness term. What do you observe? Can you explain the physics that give rise to what you observe?
* Consider the SAVI index (Soil Adjusted Vegetation Index) $ SAVI = (1-L)\cdot(NIR - RED)/(NIR+RED+L)$, where $L=0.5$
    * What is the role of $L$?
* Add some extra nuisance parameters such as "cm" and see how the relationship changes
* Try different indices and change the imaging geometry. Do you notice any patterns?
* These indices only use the red/nir bands, but what does this say about the uniqueness of using an index to retrieve a given parameter?

You can have a look at the `canopy_vi_exp` function.  It has a fairly comprehensive argument list, but this has been wrapped in a GUI below. Note that the GUI will execute the simulation whenever you change any of the widgets, and that the simulation and plot generation takes a few seconds to run!


In [4]:
canopy_vi_exp_gui()


Output()

## Exploring the MTCI (MERIS Terrestrial Chlorophyll Index)

The MTCI is a vegetation index that was developed for data from the MERIS sensor onboard the ENVISAT platform (S3/OLCI is the evolution of the MERIS sensor), and that relates surface reflectance to canopy chlorophyll content. The index is defined as the ratio of reflectance differences between MERIS bands 10 and 9 to reflectance differences between bands 9 and 8. 

$$
MTCI = \frac{R_{753}-R_{705}}{R_{705}-R_{681}},
$$
where $R_{x}$ indicated reflectance for a waveband whose centre wavelength is located at $x$. In this experiment, we will use PROSAIL to look at the robustness of such an index. We note that the bandwidth of these three bands is 7.5, 10 and 7.5 $nm$.

* A first experiment will be to determine the strength of the relationship just sweeping over the chlorophyll concentration while keeping all other parameters fixed. Try different regions of parameters space (e.g. set ``x0`` to some other value).
* As a second experiment, add some *nuisance parameters*: e.g. add a random variation variation to other parameters other than $C_{ab}$. The variation in this parameters is controlled by the ``minvals`` and ``maxvals`` dictionaries. Comment on the results, and in particular, in the robustness of the regression.
* A third experiment will also add some noise using the ``noise_level`` parameters. We can assume that ``noise_level`` is the standard deviation of some additive Gaussian noise. Values of 0.01 are probably optimistic for the type of atmospheric correction we typically get.
* Finally, observe the changes in the $MTCI-C_{ab}$ relationship for different acquisition geometries.




In [6]:
canopy_mtci_exp_gui()


Output()

## So what have we learned?

In the previous examples we have explored the PROSAIL model. Some of the things you should have noticed are:

* Parameters relating to particular pigments tend to have well defined spectral ranges where they have a large effect.
* Structural parameters such as LAI, or ALA have an influence throughout the whole spectrum
* Soil plays an important role. Even in high canopy cover areas, due to the high single scattering albedo of leaves in the NIR region, the soil signal is clearly present in the canopy reflectance.
* Angles (sun and sensor acquisition geometry) play an important role in the measured reflectance.
* Vegetation indices can be used to relate reflectance in a couple of bands to some parameters. However, when one explores the typical variation of the problem, one finds that their predictive power is limited and that other "nuisance parameters" reduce their ability to uniquely define the target variable
* It is possible to train indices with an RT model for a reduced set of conditions, but in these cases, it may be more sensible to use other non-linear regression methods.

The above comments suggest that going from canopy reflectance to a set of parameters (the so-called "inverse problem") is complicated. Effects resulting in a spectral overlap, added noise (either from the sensor, or from e.g. residual atmospheric corrections), and other confounding parameters, coupled with the limited spectral and temporal sampling make inversions hard.

The next notebooks will look at ways to improve on this.

<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.